In [ ]:
from notebooks import *
sg.init(app=None)

INFO     [17:25:51.992] [ 5049]  53 logging_/init_logging: {logging_yaml: None}

INFO     [17:25:52.283] [ 5049]  29 api.server_globals/init

•

•

INFO     [17:25:52.997] [ 5049]  31 api.server_globals/init: done

In [ ]:

def opt(*args, **kwargs):
    verbose_keys = ['cache', 'audio']
    verbose(*args, **{k: v for k, v in kwargs.items() if k in verbose_keys})
    par(**{k: v for k, v in kwargs.items() if k not in verbose_keys})

def par(**kwargs):
    kwargs = kwargs or dict(
        # use=None,                       # No par, no progress bars (easier to see audio read/write and cache hit/miss)
        # use='sync',                     # No par, cooperative progress bars (no lines printing over each other)
        use='dask', scheduler='threads',  # Par, uncooperative progress bars (lines print over each other)
    )
    config['api']['recs']['progress_kwargs'] = kwargs

def verbose(
    both=None,       # Shorthand, e.g. verbose('warn') = verbose(cache='warn', audio='warn')
    *,
    cache='debug',   # Show cache hit/miss lines # WARNING Frequent hangs during bigger xc_similar_html calls...
    # cache='info',  # Show cache hit/miss chars # WARNING (same)
    # cache='warn',  # Quiet
    audio='debug',   # Show read/write
    # audio='info',  # Show write
    # audio='warn',  # Quiet
):
    if both:
        cache = both
        audio = both
    memory.log.level = cache
    log_levels({'load': audio.upper()})

def n_recs_by_sp_quality(df):
    return (df
        # Count recs by (species, quality)
        .pipe(df_remove_unused_categories)
        .assign(n=1).groupby(['species', 'quality']).n.sum().reset_index()
        # Strip and restore .species cat around .pivot_table to workaround category error when adding .total
        .pipe(df_cat_to_str)
        # Pivot
        .pivot_table(index='species', columns='quality', values='n', fill_value=0, aggfunc='sum',
            margins=True, margins_name='total',  # FIXME 'total' ends up as NaN because we cat .species
        )
        # Restore .species cat
        .reset_index().astype({'species': metadata.species.df.shorthand.dtype}).sort_values('species').set_index('species')
        # Drop columns.name from .pivot_table ('quality')
        .T.pipe(df_set_index_name, None).T
        # Add .total
        # .pipe(df_assign_first, total=lambda df: df.sum(axis=1))
        .pipe(df_reorder_cols, first=['total'])
        # Sort
        .sort_values('total', ascending=False)
    )

In [ ]:
# How many recs per (species, quality)?
(sg.xc_meta
    # [lambda df: df.species.isin(['SNGO', 'HOFI', 'GWTE', 'YHBL'])]  # Faster dev
    .pipe(n_recs_by_sp_quality)
    .iloc[:20]
)

,total,A,B,C,D,E,no score
species,,,,,,,
NaN,35233,12109,14928,5434,1168,301,1293
RECR,659,94,281,209,63,12,0
SOSP,648,251,246,116,19,8,8
BEWR,522,168,252,69,3,0,30
SPTO,521,234,198,70,8,2,9
FOSP,501,267,153,59,14,1,7
RWBL,497,142,247,79,19,5,5
HOWR,476,163,219,55,13,2,24
AMRO,443,140,205,75,12,8,3


In [ ]:
opt('warn', use='dask', scheduler='threads')  # Quiet + par + progress bars
with print_mem_delta(collect_before=True, collect_after=False):
    display(
        xc_similar_html(
            sort='d_fc',
            sp_cols='species',
            view=False,  # Disable html view else n_recs_by_sp_quality gets junk .species values

            # XXX Before to_paths_sliced fix
            # Timing: quiet + par + progress bars + cache hits
            # xc_id=381417, n_total=None, n_sp=1,  sample_r=1,     # t[1.3s] m[   0m] n_recs[    2] p[audio  .1                       spectro  .1 plot  .1]
            # xc_id=381417, n_total=None, n_sp=1,  sample_r=None,  # t[2.4s] m[  35m] n_recs[   33] p[audio  .9                       spectro  .1 plot  .5]
            # xc_id=381417, n_total=None, n_sp=2,  sample_r=None,  # t[4.7s] m[  88m] n_recs[   77] p[audio  .2 slice  .2 persist 2.4 spectro  .2 plot 1.2]
            # xc_id=381417, n_total=None, n_sp=10, sample_r=None,  # t[ 40s] m[3500m] n_recs[  718] p[audio 1.9 slice 1.4 persist  20 spectro 1.2 plot  13]
            # xc_id=381417, n_total=None, n_sp=20, sample_r=None,  # t[118s] m[5300m] n_recs[ 1817] p[audio   7 slice   4 persist  58 spectro   3 plot  33]
            # xc_id=381417, n_total=None, n_sp=30, sample_r=None,  # t[172s] m[2600m] n_recs[ 2562] p[audio 9.4 slice   6 persist  92 spectro   4 plot  45]
            # xc_id=381417, n_total=None, n_sp=40, sample_r=None,  # t[242s] m[2650m] n_recs[ 3880] p[audio  21 slice  12 persist 105 spectro 6.4 plot  73]
            # xc_id=381417, n_total=None, n_sp=50, sample_r=None,  # t[  ?s] m[   ?m] n_recs[    ?] p[] FIXME Seemed not mem safe? ^C'd

            # Focus: Ignore O(n) plot_slice calls (more like O(10–100) in practice)
            # xc_id=381417, n_total=1,    n_sp=2,  sample_r=None,  # t[4.3s] m[  31m] n_recs[    1] p[audio  .2 slice  .2 persist 2.1 spectro  .1 plot  .1]

            # Retry after to_paths_sliced fix
            #   - NOTE to_paths_sliced happens twice -- first pass is DF([query_rec]), which is expected and a-ok
            # xc_id=381417, n_total=1,    n_sp=2,  sample_r=None,  # t[4.2s] m[  30m] n_recs[    –] p[paths  .1 audio 1.6 spectro  .1 plot  .1]
            # xc_id=381417, n_total=1,    n_sp=10, sample_r=None,  # t[ 19s] m[ 891m] n_recs[    –] p[paths  .1 audio  14 spectro  .1 plot  .1]
            # ...

            # TODO TODO More "Retry" measurements to compare vs. "Before to_paths_sliced fix"
            # TODO TODO Then think about the new .audio bottleneck -- is it essential, or a bug?
            # TODO TODO Then continue [copied verbatim from notes as Mon am refresher -- delete after reading to avoid duplication]:
            #   - [ ] How to kill the sample_r hack? Blocking learning the model's behaviors from using /similar (b/c random sample per sp)
            #     - cache/joblib/ vs. sqlite?
            #       - Is warming all the caches the way to go? What params need pinning to make that work (e.g. audio_s)? How slow to
            #         recompute when the model updates?
            #       - Would it be any simpler/faster to precompute a different representation than the joblib cache? e.g. a big sqlite
            #         file, or a big df pkl? How to maintain incrementalism, and is that still helpful?
            #     - [.] Disable sample_r to understand what unsampled operation would look like
            #     - [.] Grok state of read/write ops (local is fine), to answer whether we can make it faster without e.g. sqlite overhaul
            #     - [ ] Overhaul rebuild_cache "Cache 10s slices" section to reuse api.recs functions instead of doing it itself (duh)
            #         - e.g. sliced .audio and .feat are cached, but plot_slice isn't because we're mistakenly caching .spectro in rebuild_cache (oops)

        )
        .pipe(n_recs_by_sp_quality)
    )

INFO     [17:28:52.790] [ 5049] 831 datasets/xc_meta_to_raw_recs: Loading xc.metadata -> xc_raw_recs (.audio, more metadata)... [slower]

INFO     [17:28:52.799] [ 5049] 816 datasets/xc_meta_to_paths: Converting xc_meta -> xc_paths...

to_paths_sliced: [

] |   0% Completed |  0.0s

to_paths_sliced: [

...

plot_slice: [

################################################################################

] | 100% Completed |  0.1s

,total,A
species,,
NaN,1,1
YHBL,1,1


[mem_delta] {
  'rss': '323056 KB',
  'vms': '326164 KB',
  'pfaults': '1616 KB',
  'pageins': '0 KB',
  'uss': '304268 KB'
}

In [ ]:
# Q: Why is rec_audio_persist a bottleneck? -- shouldn't it be a noop?
#   - XXX Fixed!

# opt('warn', use='dask', scheduler='threads')  # Quiet + par + progress bars
opt('debug', use='sync')  # Verbose + sync + no progress bars

xc_similar_html(sort='d_fc', sp_cols='species',

    # xc_id=381417, n_total=5, n_sp=2,  sample_r=1,  # No rec_audio_persist
    # xc_id=381417, n_total=5, n_sp=16, sample_r=1,  # No rec_audio_persist
    # xc_id=381417, n_total=5, n_sp=17, sample_r=1,  # Yes rec_audio_persist [old behavior]

    xc_id=381417, n_total=5, n_sp=20, sample_r=1,

)